# Exploring Anchoring
This notebook is for doing initial explorations of how to investigate anchoring / in general conduct psychological experiments on large language models

In [6]:
import openai
import json
import random
import numpy as np
import asyncio
import pandas as pd
import re

In [32]:
def read_json(filename: str) -> dict:
    with open(filename) as f:
        return json.load(f)

        
def get_api_key(config, keyname: str = "goose_api") -> str:
    """
    Gets the API key from the config file.
    """
    return read_json(config)[keyname]
    
def authenticate_goose(config) -> None:
    """
    Authenticates with the goose API.
    """
    api_key = get_api_key(config, keyname="goose_api")
    openai.api_key = api_key
    openai.api_base = "https://api.goose.ai/v1"


def authenticate_openai(config) -> None:
    """
    Authenticates with the goose API.
    """
    api_key = get_api_key(config, keyname="openai_api")
    openai.api_key = api_key
    openai.api_base = "https://api.openai.com/v1"

def generate_prompt(
    prompt: str, model_name: str = "gpt-neo-125m", max_tokens: int = 75, temperature=0.9
) -> str:
    """
    Generates a prompt using a model from EleutherAI.
    """
    return openai.Completion.create(
        prompt=prompt,
        engine=model_name,
        max_tokens=max_tokens,
        temperature=temperature,
    )["choices"][0]["text"]

def find_first_num(s: str):
    """
    Finds first integer or floating point in string
    """
    try: 
        return re.findall(r"[-+]?(?:\d*\.\d+|\d+)", re.sub(r'[,]','',s))[0]
    except IndexError:
        return None


In [36]:
questions = [
    "Length of Mississippi River (in miles)",
    "Height of Mount Everest (in feet)",
    "Amount of meat eaten per year by average American (in pounds)",
    "Distance from San Francisco to New York City (in miles)",
    "Height of tallest redwood (in feet)",
    "Number of United Nations members",
    "Number of female professors at the University of California, Berkeley",
    "Population of Chicago (in millions)",
    "Year telephone was invented",
    "Average number of babies born per day in the United States",
    "Maximum speed of house cat (in miles per hour)",
    "Amount of gas used per month by average American (in gallons)",
    "Number of bars in Berkeley, CA",
    "Number of state colleges and universities in California",
    "Number of Lincoln's presidency"
]

context = """Q: Height of Taj Mahal (in feet).
A: 239.5.

Q: """

post_script = """.
A:"""

estimates_calibration = []
repetitions = 1
authenticate_goose("../config.json")
engines = ["gpt-j-6b", "gpt-neo-20b", "fairseq-13b"]

for q in questions:
    prompt = context + q + post_script
    for engine in engines:
        row = {"engine": engine, "max_tokens": 30, "temperature": 0.7,
            "prompt": prompt, "stream": False, "question": q, "n": repetitions}

        completion = openai.Completion.create(**row)
        completion_text = ""
        row["question"] = q
        for c in completion["choices"]:
            completion_text = dict(c)["text"]
            row["answer"] = completion_text

            estimates_calibration.append(row)

authenticate_openai("../config.json")
engines = ["text-davinci-002", "text-ada-001"]

for q in questions:
    prompt = context + q + post_script
    for engine in engines:
        row = {"engine": engine, "max_tokens": 30, "temperature": 0.7,
            "prompt": prompt, "stream": False, "n": repetitions}

        completion = openai.Completion.create(**row)
        completion_text = ""
        row["question"] = q
        for c in completion["choices"]:
            completion_text = dict(c)["text"]
            row["answer"] = completion_text
            estimates_calibration.append(row)

df = pd.DataFrame.from_records(estimates_calibration)
df.to_csv("output/estimates.csv")

              engine  max_tokens  temperature  \
0           gpt-j-6b          30          0.7   
1        gpt-neo-20b          30          0.7   
2        fairseq-13b          30          0.7   
3           gpt-j-6b          30          0.7   
4        gpt-neo-20b          30          0.7   
..               ...         ...          ...   
70      text-ada-001          30          0.7   
71  text-davinci-002          30          0.7   
72      text-ada-001          30          0.7   
73  text-davinci-002          30          0.7   
74      text-ada-001          30          0.7   

                                               prompt  stream  \
0   Q: Height of Taj Mahal (in feet).\nA: 239.5.\n...   False   
1   Q: Height of Taj Mahal (in feet).\nA: 239.5.\n...   False   
2   Q: Height of Taj Mahal (in feet).\nA: 239.5.\n...   False   
3   Q: Height of Taj Mahal (in feet).\nA: 239.5.\n...   False   
4   Q: Height of Taj Mahal (in feet).\nA: 239.5.\n...   False   
..                   

In [47]:
# Log nums away from the base value
log_nums = [-100000, -10000, -1000, -100, -50, -10, -5, -2, 0, 2, 5, 10, 50, 100, 1000, 10000, 100000]

# Load default numbers
estimates = pd.read_csv("output/estimates.csv")
estimates["answer_num"] = [find_first_num(answer) for answer in estimates["answer"]]

# Set context
context = """Q: Height of Taj Mahal (in feet).
A: 239.5.

Random number: """

post_number = """.
Q: """

post_script = """.
A:"""

experimental = []
repetitions = 1

# Set engines
authenticate_goose("../config.json")
engines = ["gpt-j-6b"]

# Run machine
for q in questions:
    test_numbers =  [float(estimates[estimates["question"] == q].iloc[0]["answer_num"]) + num for num in log_nums]
    print(test_numbers)
    
    for num in test_numbers:
        prompt = context + str(num) + post_number + q + post_script
        for engine in engines:
            row = {"engine": engine, "max_tokens": 30, "temperature": 0.7,
                "prompt": prompt, "stream": False, "n": repetitions}

            completion = openai.Completion.create(**row)
            completion_text = ""
            row["question"] = q
            row["anchor"] = num
            row["calibration"] = float(estimates[estimates["question"] == q].iloc[0]["answer_num"])
            row["anchor_diff"] = num - row["calibration"]

            for c in completion["choices"]:
                completion_text = dict(c)["text"]
                row["answer"] = completion_text
                estimates_calibration.append(row)
    break

df = pd.DataFrame.from_records(estimates_calibration)
df.to_csv("output/test_out.csv")
print(df)

[-99267.0, -9267.0, -267.0, 633.0, 683.0, 723.0, 728.0, 731.0, 733.0, 735.0, 738.0, 743.0, 783.0, 833.0, 1733.0, 10733.0, 100733.0]
          engine  max_tokens  temperature  \
0       gpt-j-6b          30          0.7   
1    gpt-neo-20b          30          0.7   
2    fairseq-13b          30          0.7   
3       gpt-j-6b          30          0.7   
4    gpt-neo-20b          30          0.7   
..           ...         ...          ...   
121     gpt-j-6b          30          0.7   
122     gpt-j-6b          30          0.7   
123     gpt-j-6b          30          0.7   
124     gpt-j-6b          30          0.7   
125     gpt-j-6b          30          0.7   

                                                prompt  stream  \
0    Q: Height of Taj Mahal (in feet).\nA: 239.5.\n...   False   
1    Q: Height of Taj Mahal (in feet).\nA: 239.5.\n...   False   
2    Q: Height of Taj Mahal (in feet).\nA: 239.5.\n...   False   
3    Q: Height of Taj Mahal (in feet).\nA: 239.5.\n...   False 

In [5]:
N = 10
log_nums = np.logspace(3,5,num=10, base=10, dtype=int)
engines = ['gpt-neo-20b', 'fairseq-13b', 'convo-6b', 'gpt-j-6b']
answer_dict = dict.fromkeys(log_nums)
for engine in engines:
    for num in log_nums:
        answer = asyncio.gather(*[get_answer(num, engine) for _ in range(N)])
        answer_dict[str(num)+engine] = answer

In [236]:
answer_dict = asyncio.run(run(engines=['convo-6b']))

RuntimeError: asyncio.run() cannot be called from a running event loop

In [235]:
answer_dict

<coroutine object run at 0x000002134EF13AC0>

In [214]:
newer_dict = {k: [find_first_num(i) for i in v] for k, v in new_dict.items()}

df = pd.concat({k: pd.Series(v) for k, v in newer_dict.items()}).reset_index().dropna()
df.columns = ['num', 'idx', 'answer']
df.to_csv('output/run2.csv')

TypeError: 'builtin_function_or_method' object is not iterable

In [200]:
new_dict = {k: v.result for k, v in answer_dict.items() if v is not None and type(v) is not str}
newer_dict = {k: [find_first_num(i) for i in v] for k, v in new_dict.items()}
df = pd.concat({k: pd.Series(v) for k, v in newer_dict.items()}).reset_index().dropna()
df.columns = ['num', 'idx', 'answer']
df.to_csv('output/run2.csv')

AttributeError: 'builtin_function_or_method' object has no attribute 'result'

In [ ]:


for engine in engines:
    for idx, num in enumerate(num_range):
        for cur_completion in range(completions_per_question):
            prompt = f"Random number: {num}\n{test_question}: "

            completions.append(openai.Completion.create(
                    engine=engine,
                    prompt=prompt,
                    max_tokens=75,
                    temperature=0.9,
                    stream=True))

            for i, c in enumerate(completions[-1]):
                meta.append([num, prompt, c.choices[0].text, engine])